In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk
from os.path import join, basename, isdir
sys.path.append(str(Path(getcwd()).parent.parent))

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import codecs
import numpy as np
from glob import glob
from epiweeks import Week

In [ ]:
from bulletin import default_input, default_output, tables_path, root
default_output = join(default_output,'vacinacao')

from bulletin.systems import Notifica, CasosConfirmados, eSUS, Sivep
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import normalize_text, normalize_hash, date_hash

from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
vac = eSUS('vacinados')
vac.load()
vac.df

In [ ]:
vac.df.columns

In [ ]:
# tables = dict([(Path(x).stem,pd.read_csv(join(tables_path,x))) for x in glob(join(tables_path,"*.csv"))])
# tables['faixa_etaria_populacao']

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
# Quantas pessoas já tomaram a D2 e estão há mais de 5 meses aptos para tomar a dose adicional. 
# Quantitativos e relação para PR, RS e Município.

In [ ]:
days_after = 5*30
archive = join(default_output, 'demanda_vacinacao_01.xlsx')


writer = pd.ExcelWriter(archive,
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

In [ ]:
list_search = ['1ª Dose,2ª Dose,NA', 'NA,2ª Dose,NA']

vac01 = vac.df.loc[vac.df['situacao'] != 'inconsistencia']
vac01 = vac01.loc[vac01['doses_aplicadas'].isin(list_search)]
vac01 = vac01.loc[vac01['dias_apos_ultima_dose'] >= days_after]

vac01 = pd.merge(vac01.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','macro','rs','uf','municipio','regional']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
vac01 = vac01.loc[vac01['rs'] != 99]

In [ ]:
tabela_rs = vac01.groupby(['macro', 'rs'])[['paciente']].count().reset_index().rename(columns={'rs':'RS', 'macro':'MACRO', 'paciente':'APTOS'})
tabela_rs.loc['TOTAL', ['APTOS']] = tabela_rs[['APTOS']].sum()
# tabela_rs.to_excel(writer, 'quantitativos_RS', index=False)
tabela_rs

In [ ]:
tabela_mun = vac01.groupby(['rs', 'municipio'])[['paciente']].count().reset_index().rename(columns={'municipio':'MUNICÍPIO', 'rs':'RS', 'paciente':'APTOS'})
tabela_mun.loc['TOTAL', ['APTOS']] = tabela_mun[['APTOS']].sum()
# tabela_mun.to_excel(writer, 'quantitativos_MUN', index=False)
tabela_mun

In [ ]:
writer.save()